
# MI alapú alkalmazásfejlesztés

<a target="_blank" href="https://colab.research.google.com/github/reguly/aiappdev/blob/main/AppDevelopment.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


In [2]:
%%capture
# openai python csomag telepítése
!pip install openai
# grafikus felülethez gradio
!pip install gradio
# egyéb később használt csomagok
!pip install requests
!pip install beautifulsoup4
!pip install markdownify

In [3]:
import os
os.environ['OPENAI_API_KEY'] = '' #!!!! itt kell megadni az API kulcsot

In [ ]:
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {"role": "system", "content": "Egy versíró művész vagy, aki hétköznapi helyzetek alapján ír színes, elgondolkodtató verseket."},
    {"role": "user", "content": "Írj egy verset a hétvégi korán kelés fájdalmáról."}
  ]
)
print("A válasz: \n\n")
print(completion.choices[0].message.content)

In [ ]:
print("A teljes válasz: \n\n")
print(completion.to_json())

## Chatbot fejlesztés
Használjuk az openai chat completions API-t egy chatbot implementációjára. Az input() függvénnyel a user-től bekérjünk szöveget. messages tömbbe tároljuk az oda-vissza üzeneteket (a megfelelő role user/assistant megjelölésekkel). Addig ismételjük amíg a user válasza nem "exit".

In [ ]:
# prompt: now create a simple gradio chat interface out of this. use gradio's ChatInterface. Consider that The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.

import gradio as gr
import os
from openai import OpenAI

client = OpenAI()


def chat_with_openai(message, history):
    # Add the user's message to the history
    history.append({"role": "user", "content": message})

    # Send the conversation history to OpenAI's Chat Completion API
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=history
    )

    # Extract the assistant's response from the API response
    assistant_message = response.choices[0].message.content

    # Add the assistant's response to the history
    history.append({"role": "assistant", "content": assistant_message})
    print(history)

    return assistant_message

# Create a Gradio ChatInterface
demo = gr.ChatInterface(
    fn=chat_with_openai,
    type="messages",
    title="OpenAI Chatbot"
)

# Launch the interface
demo.launch()

## Strukturált válasz


In [ ]:
from openai import OpenAI
client = OpenAI()

response = client.chat.completions.create(
  model="gpt-4o-mini",
  response_format={ "type": "json_object" },
  messages=[
    {"role": "system", "content": "You are a helpful assistant designed to output JSON."},
    {"role": "user", "content": "Who won the world series in 2020?"}
  ]
)

import json
my_dict = json.loads(response.choices[0].message.content)
print("Teljes válasz python dictionary:")
print(my_dict)
print(json.dumps(my_dict, indent=4))
print("\n")
print(f'A válasz: a győztes a {my_dict["winner"]} csapat volt {my_dict["year"]}-ban.')



In [ ]:
student_1_description = "David Nguyen is a sophomore majoring in computer science at Stanford University. He is Asian American and has a 3.8 GPA. David is known for his programming skills and is an active member of the university's Robotics Club. He hopes to pursue a career in artificial intelligence after graduating."

# A simple prompt to extract information from "student_description" in a JSON format.
prompt_1 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_1_description}
'''

# Generating response back from gpt-4o-mini
openai_response = client.chat.completions.create(
    model = 'gpt-4o-mini',
    response_format={ "type": "json_object" },
    messages = [{'role': 'user', 'content': prompt_1}]
)

print(openai_response.choices[0].message.content)



## Weblap készítés adatok megjelenítéséhez
Python Flask szerver
Mivel colab-ban futunk, meg kell oldani, hogy elérhető legyen a szerver számunkra is. Erre való az ngrok. Ha saját gépen dolgozunk, elég a flask.

In [ ]:
!pip install flask-ngrok
!pip install pyngrok
from pyngrok import ngrok, conf
conf.get_default().auth_token = '' #NGROK API KEY

In [ ]:
import os
import threading

from flask import Flask
from pyngrok import ngrok

app = Flask(__name__)
port = "5000"

# Open a ngrok tunnel to the HTTP server
public_url = ngrok.connect(port).public_url
print('Itt lehet csatlakozni: '+public_url)


# Update any base URLs to use the public ngrok URL
app.config["BASE_URL"] = public_url

# Define Flask routes
@app.route("/")
def index():
    return "Hello from Colab!"

# Start the Flask server in a new thread
app.run(debug=True)

## Weblapok letöltése, értelmezése

In [ ]:
import markdownify
import requests
from bs4 import BeautifulSoup

def beautifulsoup_web_scrape_url(url):
  headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36'}
  response = requests.get(url, headers=headers)
  soup = BeautifulSoup(response.content, 'html.parser')
  return str(soup)

url = "https://itk.ppke.hu/"

html_data = beautifulsoup_web_scrape_url(url)

# convert html to markdown
markdown_data = markdownify.markdownify(html_data, heading_style="ATX")

In [ ]:
print("Eredeti HTML adat: ")
print(html_data)

In [ ]:
print("Markdown adat: ")
print(markdown_data)

In [ ]:
# markdown formázott megjelenítése
from IPython.display import display, Markdown, Latex
display(Markdown(markdown_data))

## Függvényhívás


In [ ]:
# diák adatainak feldolgozása - valóságban egy adatbázisba mentene
def submit_student_info(name, major, school, grades, club):
    print(f"Student Name: {name}")
    print(f"Major: {major}")
    print(f"School: {school}")
    print(f"Grades: {grades}")
    print(f"Club: {club}")

# fentebbi függvény LLM számára érthető leírása
student_custom_functions = [
    {
        'name': 'submit_student_info',
        'description': 'Submit student information for processing',
        'parameters': {
            'type': 'object',
            'properties': {
                'name': {
                    'type': 'string',
                    'description': 'Name of the person'
                },
                'major': {
                    'type': 'string',
                    'description': 'Major subject.'
                },
                'school': {
                    'type': 'string',
                    'description': 'The university name.'
                },
                'grades': {
                    'type': 'integer',
                    'description': 'GPA of the student.'
                },
                'club': {
                    'type': 'string',
                    'description': 'School club for extracurricular activities. '
                }

            }
        }
    }
]

In [ ]:
from openai import OpenAI
import json
client = OpenAI()

# diákok "CV"-i
student_2_description="Ravi Patel is a sophomore majoring in computer science at the University of Michigan. He is South Asian Indian American and has a 3.7 GPA. Ravi is an active member of the university's Chess Club and the South Asian Student Association. He hopes to pursue a career in software engineering after graduating."
student_1_description = "David Nguyen is a sophomore majoring in computer science at Stanford University. He is Asian American and has a 3.8 GPA. David is known for his programming skills and is an active member of the university's Robotics Club. He hopes to pursue a career in artificial intelligence after graduating."

prompt1 = '''
I need to submit structured student information for processing to a database. This is the body of text to extract the information from:
'''

student_description = [prompt1+student_1_description,prompt1+student_2_description]
for i in student_description:
    response = client.chat.completions.create(
        model = 'gpt-4o-mini',
        messages = [{'role': 'user', 'content': i}],
        functions = student_custom_functions,
        function_call = 'auto'
    )

    # válasz JSON alapú értelmezése
    print(response.model_dump())
    # függvényhívás paramétereinek kinyerése
    json_response = json.loads(response.choices[0].message.function_call.arguments)
    print(json_response)
    # függvény meghívása
    submit_student_info(**json_response)

{'id': 'chatcmpl-AIsR1RR1mSYpwTCKhx9izJFLYBsBI', 'choices': [{'finish_reason': 'function_call', 'index': 0, 'logprobs': None, 'message': {'content': None, 'refusal': None, 'role': 'assistant', 'function_call': {'arguments': '{"name":"David Nguyen","major":"computer science","school":"Stanford University","grades":3.8,"club":"Robotics Club"}', 'name': 'submit_student_info'}, 'tool_calls': None}}], 'created': 1729062515, 'model': 'gpt-4o-mini-2024-07-18', 'object': 'chat.completion', 'service_tier': None, 'system_fingerprint': 'fp_e2bde53e6e', 'usage': {'completion_tokens': 39, 'prompt_tokens': 168, 'total_tokens': 207, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}}
{'name': 'David Nguyen', 'major': 'computer science', 'school': 'Stanford University', 'grades': 3.8, 'club': 'Robotics Club'}
Student Name: David Nguyen
Major: computer science
School: Stanford University
Grades: 3.8
Club: Robo